In [90]:
import statsmodels.api as sm
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn import preprocessing


# 2
data_set = sm.datasets.get_rdataset('airquality')
# df = pd.read_csv('https://dl.dropboxusercontent.com/u/6044937/Data%20Sets/airquality.csv', index_col=0)
# print data_set.data

# 3

data_df = data_set.data
# data_set.dropna(inplace=True)
data_df_cleaned = data_df.dropna()
scaler = preprocessing.StandardScaler().fit(data_df_cleaned)
print scaler
# print data_df_cleaned

# 4
# ozone_train = []
# for data in data_df['Ozone']:
# 	ozone_train.append([data])
    
# solarR_train = []
# for data in data_df['Solar.R']:
# 	solarR_train.append([data])

ozone = data_df_cleaned[['Ozone']]
othersVariable = data_df_cleaned.drop('Ozone', 1)

# Create linear regression object
regr = linear_model.LinearRegression()
# Train the model using the training sets
regr.fit(othersVariable, ozone)

coef = regr.coef_
intercept = regr.intercept_

print 'Coef: ' + str(coef)
print 'Intercept: ' + str(intercept)

ozoneP = regr.predict(othersVariable)

# print 'ozonePredict: ' + str(ozoneP)

residual = np.subtract(ozone, ozoneP)
# print residual

sigmaCarre = (np.sum(residual*residual) / (len(residual) - 1))[0]

print sigmaCarre

# 6

sigma = np.sqrt(sigmaCarre)
confidence = .99

confidenceCoeff = .99 / 2.0



StandardScaler(copy=True, with_mean=True, with_std=True)
Coef: [[ 0.05027432 -3.31844386  1.89578642 -3.03995664  0.27387752]]
Intercept: [-64.1163211]
415.299336129
